In [18]:
import pandas as pd
import json
from transformers import LlamaTokenizer
from tqdm import tqdm

In [19]:
data = json.load(open('tmp5.json'))
log = json.load(open('log.json'))

In [20]:
def DataProcess(data,index,log,labels_path=None):
    
    print('\n\nImporting and filtering database...')
        
    ints_str = '0123456789-#[]' # characters that usually main categories don't start with
    notes = data
    section = log[index][0]['section_used']
    print('\n\nSplitting each note into sections:\n\n')
    
    notes_sections = {}
    
    for note_index in tqdm(range(notes.shape[0])):
        
        note = notes['text'][note_index].replace('\n\n\n\n','\n').replace('\n\n\n','\n').replace('     ','\n')
        paragraphs = note.split('\n')
        
        subsections, new_section = [], ' '
        for p in paragraphs:
            line = p.strip()
            if len(line)>0 and ':' in line and not (line[line.find(':')-1] in ints_str) and not(line[0] in ints_str):
                subsections.append([new_section.strip()])
                new_section = p + ' '
            else:
                new_section += p + ' '
        subsections.append([new_section])
        subsections.pop(0)

        note_sect_tit,note_sect_par = [],[]
        for sect in subsections:
            note_sect_tit += [str(*sect)[0:str(*sect).find(':')]]
            note_sect_par += [str(*sect)[str(*sect).find(':')+1:].strip()]
        note_df = pd.DataFrame({'title':note_sect_tit,'category':'','text':note_sect_par, 'label':''})
        notes_sections[notes['note_id'][note_index]] = note_df
    
    f = open(labels_path, 'r')
    obj_label = f.readlines()
    obj_label_dict = {}
    i = 0
    for s in obj_label:
        i += 1
        if '/' in s:
            buffer = s.strip('\n').lower().split('/')
            for item in buffer:
                obj_label_dict[item] = i
        else:
            obj_label_dict[s.strip('\n').lower()] = i
    f.close()

    for key in tqdm(list(notes_sections.keys())):
        buffer = 'begin_title'
        t = list(notes_sections[key]['title'])
        for idx in range(len(t)):
            for item in list(obj_label_dict.keys()):
                if item in t[idx].lower() and len(t[idx].lower())>2:
                    buffer = item
                    notes_sections[key]['category'][idx] = buffer
                    notes_sections[key]['label'][idx] = obj_label_dict[buffer]
                    break
            notes_sections[key]['category'][idx] = buffer
            notes_sections[key]['label'][idx] = obj_label_dict[buffer]

    notes_sections_output = {}
    row_id  = notes_sections.keys()
    for key in tqdm(row_id):
        buffer = ''
        note_sect_tit, note_sect_par, note_sect_lab = [], [], []
        for i in range(len(notes_sections[key]['category'])):
            if buffer != notes_sections[key]['category'][i]:
                buffer = notes_sections[key]['category'][i]
                note_sect_tit.append(buffer)
                note_sect_lab.append(notes_sections[key]['title'][i])
                note_sect_par.append(notes_sections[key]['text'][i])
            else:
                note_sect_par[-1] = note_sect_par[-1] + ' ' + notes_sections[key]['title'][i] + ' ' + notes_sections[key]['text'][i]
        note_df = pd.DataFrame({'title': note_sect_tit, 'text': note_sect_par, 'label': note_sect_lab})
        notes_sections_output[key] = note_df


    notes_sections = notes_sections_output[index]
    
    notes_sections = notes_sections[notes_sections['label'].isin(section)]
    
    return ''.join(notes_sections['label'] + ": " + notes_sections['text'])

In [21]:
def Process(data,log):
    
    for i,key in enumerate(data['data']):
        index = data['data'][i]['title']
        temp = data['data'][i]['paragraphs'][0]['context']
        temp = pd.DataFrame({'note_id':[index],'text':[temp]})
        context = DataProcess(temp,index,log,"labels.txt")
        data['data'][i]['paragraphs'][0]['context'] = context
        
    return data

In [22]:
data = Process(data,log)

100%|██████████| 1/1 [00:00<00:00, 1540.89it/s]



Importing and filtering database...


Splitting each note into sections:




Importing and filtering database...


Splitting each note into sections:




Importing and filtering database...


Splitting each note into sections:




Importing and filtering database...


Splitting each note into sections:




Importing and filtering database...


Splitting each note into sections:




Importing and filtering database...


Splitting each note into sections:




Importing and filtering database...


Splitting each note into sections:





100%|██████████| 1/1 [00:00<00:00, 796.19it/s]




Importing and filtering database...


Splitting each note into sections:




Importing and filtering database...


Splitting each note into sections:




Importing and filtering database...


Splitting each note into sections:




In [23]:
tokenizer = LlamaTokenizer.from_pretrained("decapoda-research/llama-7b-hf")
for i,key in enumerate(data['data']):
    print(len(tokenizer.tokenize(data['data'][i]['paragraphs'][0]['context'])))

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'LLaMATokenizer'. 
The class this function is called from is 'LlamaTokenizer'.


1314
1054
1384
1381
1069
679
1327
1394
1266
1108


In [24]:
with open('QA.json', 'w') as f:      
    json.dump(data, f, indent=2)

In [34]:
mimic4_qa = []
for i,key in enumerate(data['data']):
    instruction = data['data'][i]['paragraphs'][0]['context']
    print(len(tokenizer.tokenize(instruction)))
    
    for(j,question) in enumerate(data['data'][i]['paragraphs'][0]['qas']):
        #print(j)
        question = question['question']
        answer = data['data'][i]['paragraphs'][0]['qas'][j]['answers']
        mimic4_qa.append({'instruction':instruction,'input':question,'output':answer})

1314
1054
1384
1381
1069
679
1327
1394
1266
1108


In [35]:
with open('mimic4_qa.json', 'w') as f:      
    json.dump(mimic4_qa, f, indent=2)

In [36]:
len(tokenizer.tokenize("Allergies: Sulfa (Sulfonamides) / Erythromycin BaseAttending: ___.History of Present Illness: Patient is a ___ year old female status post uterine artery  fibroid embolization by ___ on ___ who was discharged ___ and  around midnight awoke with right groin pain. The patient noticed  that her right groin appeared swollen and felt slightly firm.  This area of swelling seemed to increase in size. The patient  also felt pain radiating across her lower abdomen, causing her  to \"double over in pain.\" She also felt it difficult to  straighten her right leg without pain. She called the  interventional radiology physician's assistant, who instructed  her to come to the emergency department.  In the ED, patient was found by ultrasound to have a right groin  pseudoaneurysm. Patient was admitted for thrombin injection of  pseudoaneurysm.  On the floor, patient reported right groin pain of ___. She had  no other complaints. Review of sytems (+) Per HPI (-) Denies fever, chills, night sweats, recent weight loss or  gain. Denies headache, sinus tenderness, rhinorrhea or  congestion. Denied cough, shortness of breath. Denied chest pain  or tightness, palpitations. Denied nausea, vomiting, diarrhea,  constipation or abdominal pain. No recent change in bowel or  bladder habits. No dysuria.Past Medical History: 1. Rheumatoid Arthrits 2. Irritable Bowel Syndrome 3. Bicuspid aortic valve (no TTE in chart).  Past Surgical History 1. Bilateral knee arthroscopy 2. Bilateral ___ Metatarsal surgerySocial History: ___Physical Exam:  Vitals T: 98.4 BP:96/60 P: 63 R:18 O2: 98 RA General Alert, oriented, no acute distress HEENT Sclera anicteric, MMM, oropharynx clear Neck supple, JVP not elevated, no LAD Lungs Clear to auscultation bilaterally, no wheezes, rales,  ronchi CV Regular rate and rhythm, normal S1 + S2, no murmurs, rubs,  gallops Abdomen soft, non-tender, non-distended, bowel sounds present,  no rebound tenderness or guarding, no organomegaly Ext Warm, well perfused, 2+ pulses, no clubbing, cyanosis or  edema Right Groin Appears slightly swollen, no erythema, +bruit  auscultated, +tenderness to palpation.Brief Hospital Course: This is a ___ year old female with uterine fibroid embolization  on ___ who presented with right groin pain and swelling  determined by ultrasound to be a pseudoaneurysm.  1) Pseudoaneurysm: Patient presented with psuedoaneurysm  confirmed by ultrasound that was secondary to her recent uterine  fibroid embolization on ___. On ___ patient underwent  psuedoaneurysm thrombin injection. Ultrasound on ___ indicated  pseudoaneurysm had been thrombosed with remaining femoral artery  patent. A repeat US on ___ prior to discharge also demonstrated  thrombosed right femoral artery  pseudoaneurysm and patent right common and superficial femoral  arteries and veins. Following thrombin injection patient  experienced right groin pain. She also experienced pain in her  pelvic region thought to be secondary to fibroids responding to  embolization procedure. Pain initially treated with intravenous  morphine and transitioned to oral oxycodone at time of  discharge. She was given a prescription for oxycodone 5mg ___  tabs TID as needed for pain. She was instructed not to drive or  operate heavy machinery while taking this medication. She was  also prescribed ibuprofen 800 mg PO TID to reduce inflammation  and instructed to take this medication with food and water.  Post-procedure follow up to be arranged by ___,  interventional radiology ___.  2) Constipation: Patient experienced constipation during this  admission. Last bowel movement day prior to admission. Patient  reports that she has irritable bowel syndrome with constipation  at baseline. Likely constipation worsened due to patient taking  narcotics for pain control. Patient started on bowel regimen  including colace, senna and lactulose. Patient was discharged  with a prescription for oral lactulose. She will contact her  primary care physician if she is unable to have a bowel movement  within ___ hours of discharge.    3)Rheumatoid Arthritis:Stable during this admission. Patient was  continued on outpatient regimen of prednisone and folate.  Patient had taken weekly dose of methotrexate prior to  admission.Medications on Admission: 1. Prednisone 5mg QDAY 2. Folate 1 mg QDAY 3. Methotrexate 10mg Q wk 4. Culturale (probiotic) dailyDischarge Disposition: Home"))

1314